# Обработка датасета

## Библиотеки и скрипты

In [1]:
import numpy as np
import pandas as pd
import re
from transliterate import translit, get_available_language_codes

# Функции для обработки данных
import functions as fn

# Отключаем предупреждения
import warnings
warnings.simplefilter('ignore')

## Обзор данных

In [2]:
# Загружаем датасеты без 1-й строки и делаем переиндексацию с удалением столбца index
df1 = pd.read_csv('../data/hubex.ru-ClientID-Sources.csv')[1:].reset_index(drop=True) # Источники
df2 = pd.read_csv('../data/hubex.ru-ClientID-Behavior.csv')[1:].reset_index(drop=True) # Поведение
# Удаляем дубликаты по ClientID
df1 = df1.drop_duplicates(subset='ClientID', keep="first")
df2 = df2.drop_duplicates(subset='ClientID', keep="first")

In [3]:
df1.head(5)

,ClientID,Первый источник трафика,Первый переход с сайтов,Первая поисковая система,UTM Source,UTM Content,Город,Визиты,Посетители,Отказы,Глубина просмотра,Время на сайте,Новые посетители,Дней между визитами,Вернувшиеся: 1 день,Вернувшиеся: 2‑7 дней,Достижения любой цели
0,1544792334967355948,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,562,1,0.145907,2.734875,00:05:27,0,0.000000,0.645907,0.350534,800
1,1571055035849817227,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,267,2,0.142322,3.112360,00:07:45,1,0.000000,0.546816,0.434457,302
2,1599476927749455055,Переходы по ссылкам на сайтах,hubex.cloud,Не определено,Не определено,Не определено,Санкт-Петербург,248,1,0.088710,3.733871,00:07:59,1,0.364372,0.572581,0.419355,1195
3,1596550694967369930,Переходы из поисковых систем,Не определено,Google,Не определено,Не определено,Санкт-Петербург,231,1,0.134199,2.246753,00:03:04,1,1.004348,0.480519,0.497835,639
4,1564737646183903772,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,201,2,0.024876,1.268657,00:01:28,1,1.000000,0.373134,0.572139,130


## Обработка данных

In [4]:
# Производим слияние датасетов
df = pd.merge(df1, df2, how='outer')

In [5]:
df.head(5)

,ClientID,Первый источник трафика,Первый переход с сайтов,Первая поисковая система,UTM Source,UTM Content,Город,Визиты,Посетители,Отказы,...,Дней между визитами,Вернувшиеся: 1 день,Вернувшиеся: 2‑7 дней,Достижения любой цели,Дата первого визита,Дата визита,Достигнутая цель,Путь (полный) страницы входа,Путь (полный) страницы выхода,Мобильность
0,1544792334967355948,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,562,1.0,0.145907,...,0.000000,0.645907,0.350534,800.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1571055035849817227,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,267,2.0,0.142322,...,0.000000,0.546816,0.434457,302.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1599476927749455055,Переходы по ссылкам на сайтах,hubex.cloud,Не определено,Не определено,Не определено,Санкт-Петербург,248,1.0,0.088710,...,0.364372,0.572581,0.419355,1195.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1596550694967369930,Переходы из поисковых систем,Не определено,Google,Не определено,Не определено,Санкт-Петербург,231,1.0,0.134199,...,1.004348,0.480519,0.497835,639.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1564737646183903772,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,201,2.0,0.024876,...,1.000000,0.373134,0.572139,130.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.columns

Index(['ClientID', 'Первый источник трафика', 'Первый переход с сайтов',
       'Первая поисковая система', 'UTM Source', 'UTM Content', 'Город',
       'Визиты', 'Посетители', 'Отказы', 'Глубина просмотра', 'Время на сайте',
       'Новые посетители', 'Дней между визитами', 'Вернувшиеся: 1 день',
       'Вернувшиеся: 2‑7 дней', 'Достижения любой цели', 'Дата первого визита',
       'Дата визита', 'Достигнутая цель', 'Путь (полный) страницы входа',
       'Путь (полный) страницы выхода', 'Мобильность'],
      dtype='object')

In [7]:
df.shape

(77456, 23)

In [8]:
# Удаление ненужных данных
columns = ['Посетители']
df = fn.drop_col(df, columns)

# Сегментация пользователей
df.loc[df['Дата визита'].isnull(), 'Дата визита'] = 'undefined'
# df = df[df['Дата визита'].str.contains('202.*')]
# df = df.loc[(df['Визиты'] <= 35) & (df['Достижения любой цели'] <= 100)]
# df = df.loc[(df['ClientID'] != '')]

df.head(2)

,ClientID,Первый источник трафика,Первый переход с сайтов,Первая поисковая система,UTM Source,UTM Content,Город,Визиты,Отказы,Глубина просмотра,...,Дней между визитами,Вернувшиеся: 1 день,Вернувшиеся: 2‑7 дней,Достижения любой цели,Дата первого визита,Дата визита,Достигнутая цель,Путь (полный) страницы входа,Путь (полный) страницы выхода,Мобильность
0,1544792334967355948,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,562,0.145907,2.734875,...,0.0,0.645907,0.350534,800.0,NaN,undefined,NaN,NaN,NaN,NaN
1,1571055035849817227,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,267,0.142322,3.112360,...,0.0,0.546816,0.434457,302.0,NaN,undefined,NaN,NaN,NaN,NaN


In [9]:
# Переименование столбцов
df.columns = fn.get_translite(df.columns, 'ym_')
df.columns

Index(['ym_clientid', 'ym_pervyj_istochnik_trafika',
       'ym_pervyj_perehod_s_sajtov', 'ym_pervaja_poiskovaja_sistema',
       'ym_utm_source', 'ym_utm_content', 'ym_gorod', 'ym_vizity', 'ym_otkazy',
       'ym_glubina_prosmotra', 'ym_vremja_na_sajte', 'ym_novye_posetiteli',
       'ym_dnej_mezhdu_vizitami', 'ym_vernuvshiesja_1_den',
       'ym_vernuvshiesja_2_7_dnej', 'ym_dostizhenija_ljuboj_tseli',
       'ym_data_pervogo_vizita', 'ym_data_vizita', 'ym_dostignutaja_tsel',
       'ym_put_polnyj_stranitsy_vhoda', 'ym_put_polnyj_stranitsy_vyhoda',
       'ym_mobilnost'],
      dtype='object')

**Обработка пропусков**

In [10]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77456 entries, 0 to 77455
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ym_clientid                     77456 non-null  object 
 1   ym_pervyj_istochnik_trafika     56785 non-null  object 
 2   ym_pervyj_perehod_s_sajtov      56785 non-null  object 
 3   ym_pervaja_poiskovaja_sistema   56785 non-null  object 
 4   ym_utm_source                   56785 non-null  object 
 5   ym_utm_content                  56785 non-null  object 
 6   ym_gorod                        56785 non-null  object 
 7   ym_vizity                       77456 non-null  int64  
 8   ym_otkazy                       56780 non-null  float64
 9   ym_glubina_prosmotra            56780 non-null  float64
 10  ym_vremja_na_sajte              56780 non-null  object 
 11  ym_novye_posetiteli             56785 non-null  float64
 12  ym_dnej_mezhdu_vizitami         

ym_clientid                           0
ym_pervyj_istochnik_trafika       20671
ym_pervyj_perehod_s_sajtov        20671
ym_pervaja_poiskovaja_sistema     20671
ym_utm_source                     20671
ym_utm_content                    20671
ym_gorod                          20671
ym_vizity                             0
ym_otkazy                         20676
ym_glubina_prosmotra              20676
ym_vremja_na_sajte                20676
ym_novye_posetiteli               20671
ym_dnej_mezhdu_vizitami           67235
ym_vernuvshiesja_1_den            20676
ym_vernuvshiesja_2_7_dnej         20676
ym_dostizhenija_ljuboj_tseli      20671
ym_data_pervogo_vizita            20716
ym_data_vizita                        0
ym_dostignutaja_tsel              20716
ym_put_polnyj_stranitsy_vhoda     20716
ym_put_polnyj_stranitsy_vyhoda    20716
ym_mobilnost                      20716
dtype: int64

In [11]:
# Корректировка значений
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Не определено', 'ym_pervyj_istochnik_trafika'] = 'undefined'
# ------ Для склейки заменяем пропуски на np.nan
df.loc[df['ym_pervyj_perehod_s_sajtov'] == 'Не определено', 'ym_pervyj_perehod_s_sajtov'] = np.nan
df.loc[df['ym_pervaja_poiskovaja_sistema'] == 'Не определено', 'ym_pervaja_poiskovaja_sistema'] = np.nan
df.loc[df['ym_utm_source'] == 'Не определено', 'ym_utm_source'] = np.nan
df.loc[df['ym_utm_content'] == 'Не определено', 'ym_utm_content'] = np.nan
# ------
df.loc[df['ym_gorod'].isnull(), 'ym_gorod'] = 'undefined'
df.loc[df['ym_gorod'] == 'Не определено', 'ym_gorod'] = 'undefined'
df.loc[df['ym_otkazy'].isnull(), 'ym_otkazy'] = df['ym_otkazy'].median()
df.loc[df['ym_glubina_prosmotra'].isnull(), 'ym_glubina_prosmotra'] = df['ym_glubina_prosmotra'].median()
df.loc[df['ym_vremja_na_sajte'].isnull(), 'ym_vremja_na_sajte'] = 'undefined'
df.loc[df['ym_novye_posetiteli'].isnull(), 'ym_novye_posetiteli'] = 'undefined'
df.loc[df['ym_dnej_mezhdu_vizitami'].isnull(), 'ym_dnej_mezhdu_vizitami'] = df['ym_dnej_mezhdu_vizitami'].median()
df.loc[df['ym_vernuvshiesja_1_den'].isnull(), 'ym_vernuvshiesja_1_den'] = df['ym_vernuvshiesja_1_den'].median()
df.loc[df['ym_vernuvshiesja_2_7_dnej'].isnull(), 'ym_vernuvshiesja_2_7_dnej'] = df['ym_vernuvshiesja_2_7_dnej'].median()
df.loc[df['ym_data_pervogo_vizita'].isnull(), 'ym_data_pervogo_vizita'] = 'undefined'
df.loc[df['ym_dostignutaja_tsel'].isnull(), 'ym_dostignutaja_tsel'] = 'undefined'
df.loc[df['ym_dostignutaja_tsel'] == 'Не определено', 'ym_dostignutaja_tsel'] = 'undefined'
df.loc[df['ym_put_polnyj_stranitsy_vhoda'].isnull(), 'ym_put_polnyj_stranitsy_vhoda'] = 'undefined'
df.loc[df['ym_put_polnyj_stranitsy_vyhoda'].isnull(), 'ym_put_polnyj_stranitsy_vyhoda'] = 'undefined'
df.loc[df['ym_mobilnost'].isnull(), 'ym_mobilnost'] = 'undefined' # Если не определено
df.loc[df['ym_mobilnost'] == 1.0, 'ym_mobilnost'] = 1
df.loc[df['ym_mobilnost'] == 0.0, 'ym_mobilnost'] = 0

In [12]:
# Приведение типов
df = fn.astype_col(df, ['ym_vizity'], coltype='uint8')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77456 entries, 0 to 77455
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ym_clientid                     77456 non-null  object 
 1   ym_pervyj_istochnik_trafika     56785 non-null  object 
 2   ym_pervyj_perehod_s_sajtov      5562 non-null   object 
 3   ym_pervaja_poiskovaja_sistema   3731 non-null   object 
 4   ym_utm_source                   16379 non-null  object 
 5   ym_utm_content                  16207 non-null  object 
 6   ym_gorod                        77456 non-null  object 
 7   ym_vizity                       77456 non-null  uint8  
 8   ym_otkazy                       77456 non-null  float64
 9   ym_glubina_prosmotra            77456 non-null  float64
 10  ym_vremja_na_sajte              77456 non-null  object 
 11  ym_novye_posetiteli             77456 non-null  object 
 12  ym_dnej_mezhdu_vizitami         

In [14]:
# Просмотр названий каждого канала
df['ym_pervyj_istochnik_trafika'].unique()

array(['Прямые заходы', 'Переходы по ссылкам на сайтах',
       'Переходы из поисковых систем', 'Внутренние переходы',
       'Переходы по рекламе', 'Переходы из мессенджеров',
       'Переходы из социальных сетей', 'Переходы с почтовых рассылок',
       'Переходы из рекомендательных систем', 'undefined',
       'Переходы с сохранённых страниц', nan], dtype=object)

In [15]:
# Добавляем стоимость клика по каждому каналу
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Прямые заходы', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по рекламе', 'ym_cost'] = 15
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из поисковых систем', 'ym_cost'] = 48.5
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Внутренние переходы', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по ссылкам на сайтах', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из социальных сетей', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы с почтовых рассылок', 'ym_cost'] = 2
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из рекомендательных систем', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из мессенджеров', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы с сохранённых страниц', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'].isnull(), 'ym_pervyj_istochnik_trafika'] = 'undefined'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'undefined', 'ym_cost'] = 0

In [16]:
# Переименовываем категории каналов
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Прямые заходы', 'ym_pervyj_istochnik_trafika'] = 'Browser-Traffic'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по рекламе', 'ym_pervyj_istochnik_trafika'] = 'Ad-Traffic'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из поисковых систем', 'ym_pervyj_istochnik_trafika'] = 'Search'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Внутренние переходы', 'ym_pervyj_istochnik_trafika'] = 'Site-Traffic'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по ссылкам на сайтах', 'ym_pervyj_istochnik_trafika'] = 'Sites'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из социальных сетей', 'ym_pervyj_istochnik_trafika'] = 'Social'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы с почтовых рассылок', 'ym_pervyj_istochnik_trafika'] = 'Email'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из рекомендательных систем', 'ym_pervyj_istochnik_trafika'] = 'RecSys'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из мессенджеров', 'ym_pervyj_istochnik_trafika'] = 'Messenger'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы с сохранённых страниц', 'ym_pervyj_istochnik_trafika'] = 'SavedPages'

In [17]:
df.head(5)

,ym_clientid,ym_pervyj_istochnik_trafika,ym_pervyj_perehod_s_sajtov,ym_pervaja_poiskovaja_sistema,ym_utm_source,ym_utm_content,ym_gorod,ym_vizity,ym_otkazy,ym_glubina_prosmotra,...,ym_vernuvshiesja_1_den,ym_vernuvshiesja_2_7_dnej,ym_dostizhenija_ljuboj_tseli,ym_data_pervogo_vizita,ym_data_vizita,ym_dostignutaja_tsel,ym_put_polnyj_stranitsy_vhoda,ym_put_polnyj_stranitsy_vyhoda,ym_mobilnost,ym_cost
0,1544792334967355948,Browser-Traffic,NaN,NaN,NaN,NaN,Санкт-Петербург,50,0.145907,2.734875,...,0.645907,0.350534,800.0,undefined,undefined,undefined,undefined,undefined,undefined,0.0
1,1571055035849817227,Browser-Traffic,NaN,NaN,NaN,NaN,Санкт-Петербург,11,0.142322,3.112360,...,0.546816,0.434457,302.0,undefined,undefined,undefined,undefined,undefined,undefined,0.0
2,1599476927749455055,Sites,hubex.cloud,NaN,NaN,NaN,Санкт-Петербург,248,0.088710,3.733871,...,0.572581,0.419355,1195.0,undefined,undefined,undefined,undefined,undefined,undefined,0.0
3,1596550694967369930,Search,NaN,Google,NaN,NaN,Санкт-Петербург,231,0.134199,2.246753,...,0.480519,0.497835,639.0,undefined,undefined,undefined,undefined,undefined,undefined,48.5
4,1564737646183903772,Browser-Traffic,NaN,NaN,NaN,NaN,Санкт-Петербург,201,0.024876,1.268657,...,0.373134,0.572139,130.0,undefined,undefined,undefined,undefined,undefined,undefined,0.0


In [18]:
# Слияние столбцов
df['ym_source'] = df[df.columns[2:6]].apply(lambda x: ' > '.join(x.dropna().astype(str)), axis=1)

In [19]:
# Выводим имена столцов
fn.get_idx_col(df.columns)

0 ym_clientid
1 ym_pervyj_istochnik_trafika
2 ym_pervyj_perehod_s_sajtov
3 ym_pervaja_poiskovaja_sistema
4 ym_utm_source
5 ym_utm_content
6 ym_gorod
7 ym_vizity
8 ym_otkazy
9 ym_glubina_prosmotra
10 ym_vremja_na_sajte
11 ym_novye_posetiteli
12 ym_dnej_mezhdu_vizitami
13 ym_vernuvshiesja_1_den
14 ym_vernuvshiesja_2_7_dnej
15 ym_dostizhenija_ljuboj_tseli
16 ym_data_pervogo_vizita
17 ym_data_vizita
18 ym_dostignutaja_tsel
19 ym_put_polnyj_stranitsy_vhoda
20 ym_put_polnyj_stranitsy_vyhoda
21 ym_mobilnost
22 ym_cost
23 ym_source
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [20]:
# Новый порядок столбцов
new_order = [0, 1, 23, 2, 3, 4, 5, 6, 22, 16, 17, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21]
df = df[df.columns[new_order]]

In [21]:
df.columns

Index(['ym_clientid', 'ym_pervyj_istochnik_trafika', 'ym_source',
       'ym_pervyj_perehod_s_sajtov', 'ym_pervaja_poiskovaja_sistema',
       'ym_utm_source', 'ym_utm_content', 'ym_gorod', 'ym_cost',
       'ym_data_pervogo_vizita', 'ym_data_vizita', 'ym_vizity', 'ym_otkazy',
       'ym_glubina_prosmotra', 'ym_vremja_na_sajte', 'ym_novye_posetiteli',
       'ym_dnej_mezhdu_vizitami', 'ym_vernuvshiesja_1_den',
       'ym_vernuvshiesja_2_7_dnej', 'ym_dostizhenija_ljuboj_tseli',
       'ym_dostignutaja_tsel', 'ym_put_polnyj_stranitsy_vhoda',
       'ym_put_polnyj_stranitsy_vyhoda', 'ym_mobilnost'],
      dtype='object')

In [22]:
# Корректировка значений
df['ym_source'] = df['ym_source'].str.lower() # Переводим все нижний регистр
df.loc[(df['ym_source'] == 'яндекс') | (df['ym_source'] == 'yandex.ru'), 'ym_source'] = 'yandex'
df.loc[(df['ym_source'] == '') | (df['ym_source'].isnull()), 'ym_source'] = 'undefined' # Убираем Null и пустоты
df = df[~df['ym_source'].str.contains('([0-9]{1,3}[\.]){3}[0-9]{1,3}')] # Убираем ip-адреса
df.loc[df['ym_source'] == 'мобильные%сотрудники', 'ym_source'] = 'undefined'
df.loc[df['ym_source'] == 'старт за 2 дня', 'ym_source'] = 'undefined'
df.loc[df['ym_source'] == 'яндекс > serp', 'ym_source'] = 'yandex'
# -------
df.loc[df['ym_pervyj_perehod_s_sajtov'].isnull(), 'ym_pervyj_perehod_s_sajtov'] = 'undefined'
df.loc[df['ym_pervaja_poiskovaja_sistema'].isnull(), 'ym_pervaja_poiskovaja_sistema'] = 'undefined'
df.loc[df['ym_utm_source'].isnull(), 'ym_utm_source'] = 'undefined'
df.loc[df['ym_utm_content'].isnull(), 'ym_utm_content'] = 'undefined'

In [23]:
df.head(5)

,ym_clientid,ym_pervyj_istochnik_trafika,ym_source,ym_pervyj_perehod_s_sajtov,ym_pervaja_poiskovaja_sistema,ym_utm_source,ym_utm_content,ym_gorod,ym_cost,ym_data_pervogo_vizita,...,ym_vremja_na_sajte,ym_novye_posetiteli,ym_dnej_mezhdu_vizitami,ym_vernuvshiesja_1_den,ym_vernuvshiesja_2_7_dnej,ym_dostizhenija_ljuboj_tseli,ym_dostignutaja_tsel,ym_put_polnyj_stranitsy_vhoda,ym_put_polnyj_stranitsy_vyhoda,ym_mobilnost
0,1544792334967355948,Browser-Traffic,undefined,undefined,undefined,undefined,undefined,Санкт-Петербург,0.0,undefined,...,00:05:27,0.0,0.000000,0.645907,0.350534,800.0,undefined,undefined,undefined,undefined
1,1571055035849817227,Browser-Traffic,undefined,undefined,undefined,undefined,undefined,Санкт-Петербург,0.0,undefined,...,00:07:45,1.0,0.000000,0.546816,0.434457,302.0,undefined,undefined,undefined,undefined
2,1599476927749455055,Sites,hubex.cloud,hubex.cloud,undefined,undefined,undefined,Санкт-Петербург,0.0,undefined,...,00:07:59,1.0,0.364372,0.572581,0.419355,1195.0,undefined,undefined,undefined,undefined
3,1596550694967369930,Search,google,undefined,Google,undefined,undefined,Санкт-Петербург,48.5,undefined,...,00:03:04,1.0,1.004348,0.480519,0.497835,639.0,undefined,undefined,undefined,undefined
4,1564737646183903772,Browser-Traffic,undefined,undefined,undefined,undefined,undefined,Санкт-Петербург,0.0,undefined,...,00:01:28,1.0,1.000000,0.373134,0.572139,130.0,undefined,undefined,undefined,undefined


In [24]:
df.describe()

,ym_cost,ym_vizity,ym_otkazy,ym_glubina_prosmotra,ym_dnej_mezhdu_vizitami,ym_vernuvshiesja_1_den,ym_vernuvshiesja_2_7_dnej,ym_dostizhenija_ljuboj_tseli
count,77450.000000,77450.000000,77450.000000,77450.000000,77450.000000,77450.000000,77450.000000,56779.000000
mean,10.754855,1.331026,0.324114,1.235191,1.318433,0.037865,0.031584,1.290002
std,11.131857,2.546603,0.457671,0.966847,4.713549,0.142323,0.135303,16.699739
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,1.000000,0.666667,0.000000,0.000000,0.000000
50%,15.000000,1.000000,0.000000,1.000000,0.666667,0.000000,0.000000,1.000000
75%,15.000000,1.000000,1.000000,1.000000,0.666667,0.000000,0.000000,1.000000
max,48.500000,248.000000,1.000000,46.000000,153.000000,1.000000,1.000000,2425.000000


In [25]:
df.shape

(77450, 24)

## Сохранение данных

In [26]:
# Сохраняем в файл
df.to_csv('../data/td_metrika.csv', index=False, encoding='utf-8', sep=';')